## Computer Vision

# Project 03

#### Preprocessing

Import libraries.

In [1]:
import os
import time
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from GoDec import GoDecFast

Read dataset, resize the images, create input matrix X, and create CSV file.

In [2]:
t0 = time.time()

X = []
scale_percent = 10/100
cd = '../../../../'
input_directory = 'Multimedia/Datasets/Human Actions/UoL 3D Continuous Social Activity Dataset/'
output_directory = 'Multimedia/Datasets/Human Actions/Dataset/'
video = 'Video_01'
input_path = cd + input_directory + video + '/'
output_path = cd + output_directory + video + '/'
csvfile = video + '_gray.csv'
f = open(output_path+csvfile,'w')
output_path = output_path + 'Gray/'

filenames = os.listdir(input_path)
for i,filename in enumerate(sorted(filenames)):
    img = cv.imread(input_path+filename,cv.IMREAD_GRAYSCALE)
    width = int(img.shape[1]*scale_percent)
    height = int(img.shape[0]*scale_percent)
    dim = (width,height)
    img = cv.resize(img,dim,interpolation = cv.INTER_AREA)
    cv.imwrite(output_path+filename,img)
    f.write(output_path+filename)
    f.write(",")
    f.write(video)
    f.write(",")
    f.write("\n")
    X.append(img.flatten())

X = np.array(X).T
f.close()

t1 = time.time()
print("Read data execution time: " + str(t1 - t0) + "s")

Read data execution time: 44.116095781326294s


Compute GoDec

In [3]:
rank    = 1;
card    = np.prod(X.shape);
power   = 1;
epsilon = 0.001;

t0 = time.time()
L,S,E = GoDecFast(X,rank,card,power,epsilon)
t1 = time.time()
print("GoDec execution time: " + str(t1 - t0) + "s")

GoDec execution time: 8.341811180114746s


Apply threshold and normalize sparse matrix.

In [4]:
t0 = time.time()

Min = min(S.flatten())
Max= max(S.flatten())
s = np.where(np.logical_and(S>=0,S<=Max),0,S)
s = cv.normalize(s,None,0,255,cv.NORM_MINMAX,cv.CV_8U)

t1 = time.time()
print("Threshold and normalize execution time: " + str(t1 - t0) + "s")

Threshold and normalize execution time: 2.127124786376953s


Create GoDec sparse matrix video and sparse matrix dataset.

In [5]:
cd = '../../../../'
output_directory = 'Multimedia/Datasets/Human Actions/Dataset/'
#video = 'Video_01'
output_path = cd + output_directory + video + '/'
csvfile = video + '_godec.csv'
f = open(output_path+csvfile,'w')
output_path = output_path + 'GoDec/'

t0 = time.time()

#fig = plt.figure()
#MP4 = []
for i in range(X.shape[1]):
    x = np.array(s[:,i])
    n_img = x.reshape(height,width)
    
    filename = str(i) + ".png"
    cv.imwrite(output_path+filename,n_img)
    f.write(output_path+filename)
    f.write(",")
    f.write(video)
    f.write(",")
    f.write("\n")
    
    #v = plt.imshow(n_img,cmap='gray',animated=True)
    #MP4.append([v])

#ani = animation.ArtistAnimation(fig,MP4,interval=50,blit=True,repeat_delay=1000)
#ani.save('GoDec_Resulta_S.mp4')
#plt.show()

t1 = time.time()
print("Write data execution time: " + str(t1 - t0) + "s")

Write data execution time: 0.2778933048248291s
